# Polars LazyFrameの操作
大規模データの前処理のために、遅延評価が可能なLazyFrameの基本を学ぶ。  
まだalpha版の部分(.collectのstreamingなど)があるので挙動には注意が必要。

参考：  
* https://pola-rs.github.io/polars/py-polars/html/reference/lazyframe/index.html

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import datetime

# 文字列カラムの表示文字数を50文字に設定
pl.Config.set_fmt_str_lengths(50)

polars.config.Config

# サンプルデータ作成

In [30]:
# pl.datetime_rangeで任意の範囲の時間データを作れる
col_datetime = pl.datetime_range(datetime.datetime(2022, 1, 1), datetime.datetime(2022, 4, 10, 23), interval='1h', eager=True)
df = pl.DataFrame({"DATE": col_datetime, "a": np.arange(2400), "b": list("abcaddbe" * 300)})
df.write_csv('input/sample_data_Lazy.csv')

In [74]:
df.head(3)

DATE,a,b
datetime[μs],i64,str
2022-01-01 00:00:00,0,"""a"""
2022-01-01 01:00:00,1,"""b"""
2022-01-01 02:00:00,2,"""c"""


# LazyFrameの基本
.collect()で最終的な結果を得る以外はそれほど通常のDataFrameと操作は変わらない。

In [2]:
# LazyFrameとして読み込み。この時点ではファイルの内容はほぼメモリに読み込まれない。
df_lazy = pl.scan_csv('input/sample_data_Lazy.csv')
df_lazy

<LazyFrame [3 cols, {"DATE": Utf8 … "b": Utf8}] at 0x7FB33CEBEE90>

In [5]:
# read_csvと異なり、columnsの引数はない模様。
# ヘッダーなしのcsvにヘッダーを付与して特定カラムだけ読み込みたい場合は下記のようにしてselectすればよい。
# このようにすれば.collectで実際にcsv本体をメモリに読み込む前に列の取捨選択が可能。filter付ければ行の取捨選択も事前にできる。
pl.scan_csv('input/sample_data_Lazy_headerless.csv', new_columns=['A', 'B', 'C']).select(
    ['A', 'C']
)

<LazyFrame [2 cols, {"A": Utf8, "C": Utf8}] at 0x7FB2FB05F310>

In [6]:
type(df_lazy)

polars.lazyframe.frame.LazyFrame

In [37]:
# columnsは普通に見れる。
df_lazy.columns

['DATE', 'a', 'b']

In [102]:
# .collect()を実行することで通常のDataFrameに変換できる。つまり、この時点でメモリに読み込まれる。
df_lazy.collect()


DATE,a,b
str,i64,str
"""2022-01-01T00:00:00.000000""",0,"""a"""
"""2022-01-01T01:00:00.000000""",1,"""b"""
"""2022-01-01T02:00:00.000000""",2,"""c"""
"""2022-01-01T03:00:00.000000""",3,"""a"""
"""2022-01-01T04:00:00.000000""",4,"""d"""
"""2022-01-01T05:00:00.000000""",5,"""d"""
"""2022-01-01T06:00:00.000000""",6,"""b"""
"""2022-01-01T07:00:00.000000""",7,"""e"""
"""2022-01-01T08:00:00.000000""",8,"""a"""


In [79]:
# .fetch(レコード数)で部分的にDataFrame化できるが、現状はデバッグ目的のみで使用推奨とのこと。
df_lazy.fetch(5)

DATE,a,b
str,i64,str
"""2022-01-01T00:00:00.000000""",0,"""a"""
"""2022-01-01T01:00:00.000000""",1,"""b"""
"""2022-01-01T02:00:00.000000""",2,"""c"""
"""2022-01-01T03:00:00.000000""",3,"""a"""
"""2022-01-01T04:00:00.000000""",4,"""d"""


In [84]:
# .fetchを使わずに同じことがしたければ下記のようにすればOK
df_lazy.head(5).collect()

DATE,a,b
str,i64,str
"""2022-01-01T00:00:00.000000""",0,"""a"""
"""2022-01-01T01:00:00.000000""",1,"""b"""
"""2022-01-01T02:00:00.000000""",2,"""c"""
"""2022-01-01T03:00:00.000000""",3,"""a"""
"""2022-01-01T04:00:00.000000""",4,"""d"""


In [86]:
# select、filterしてもこの時点ではレコードは返されない。
df_lazy.select(['DATE', 'b']).filter(
    pl.col('b') == 'a'
)

<LazyFrame [2 cols, {"DATE": Utf8, "b": Utf8}] at 0x7FC070567C90>

In [91]:
# .collect()を実行することでクエリが評価された結果が返される。
df_lazy.select(['DATE', 'b']).filter(
    pl.col('b') == 'a'
).collect().head(5)

DATE,b
str,str
"""2022-01-01T00:00:00.000000""","""a"""
"""2022-01-01T03:00:00.000000""","""a"""
"""2022-01-01T08:00:00.000000""","""a"""
"""2022-01-01T11:00:00.000000""","""a"""
"""2022-01-01T16:00:00.000000""","""a"""


In [122]:
%time
# パーセンタイルをstreamingで計算することは可能な模様。
# メモリに乗らない大容量データにも通用するかも。
df_lazy.select(
    pl.col('a').quantile(0.3)
).collect(streaming=True)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 7.39 µs


a
f64
720.0


In [13]:
%time
# explainで上記のクエリプランを確認するとstreamingっぽくなっている？
print( 
    df_lazy.select(
        pl.col('a').quantile(0.3)
    ).explain(streaming=True) 
)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.06 µs
 SELECT [col("a").quantile()] FROM
  --- PIPELINE

  Csv SCAN input/sample_data_Lazy.csv
  PROJECT 1/3 COLUMNS  --- END PIPELINE

    DF []; PROJECT */0 COLUMNS; SELECTION: "None"


/tmp/ipykernel_16463/2948128180.py:6: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  ).explain(streaming=True)


In [9]:
# 2つのLazyFrameをDATEでマージしてみる
df_lazy2 = pl.scan_csv('input/sample_data_Lazy_2.csv')
df_lazy2.head(5).collect()

DATE,a,b
str,i64,str
"""2022-01-01T00:00:00.000000""",0,"""axx"""
"""2022-01-01T01:00:00.000000""",1,"""bxcx"""
"""2022-01-01T02:00:00.000000""",2,"""cxcx"""
"""2022-01-01T03:00:00.000000""",3,"""axcx"""
"""2022-01-01T04:00:00.000000""",4,"""dxcx"""


In [14]:
# inner joinのクエリプラン確認
print(df_lazy.join(df_lazy2, on='DATE', how='inner').explain())

INNER JOIN:
LEFT PLAN ON: [col("DATE")]

    Csv SCAN input/sample_data_Lazy.csv
    PROJECT */3 COLUMNS
RIGHT PLAN ON: [col("DATE")]

    Csv SCAN input/sample_data_Lazy_2.csv
    PROJECT */3 COLUMNS
END INNER JOIN


In [16]:
# .collect()でマージ実行
df_lazy.join(df_lazy2, on='DATE', how='inner').collect()

DATE,a,b,a_right,b_right
str,i64,str,i64,str
"""2022-01-01T00:00:00.000000""",0,"""a""",0,"""axx"""
"""2022-01-01T01:00:00.000000""",1,"""b""",1,"""bxcx"""
"""2022-01-01T02:00:00.000000""",2,"""c""",2,"""cxcx"""
"""2022-01-01T03:00:00.000000""",3,"""a""",3,"""axcx"""
"""2022-01-01T04:00:00.000000""",4,"""d""",4,"""dxcx"""
"""2022-01-01T05:00:00.000000""",5,"""d""",5,"""dxcx"""
"""2022-01-01T06:00:00.000000""",6,"""b""",6,"""bxcx"""
"""2022-01-01T07:00:00.000000""",7,"""e""",7,"""excx"""
"""2022-01-01T08:00:00.000000""",8,"""a""",8,"""axcx"""
